In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Get Product Information from Search Page

In [45]:
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

# .get response code
r = requests.get('https://www.amazon.com/s?k=board+games&page=1', headers=headers)
content = r.content
soup = BeautifulSoup(content)

In [31]:
soup

<!--
        To discuss automated access to Amazon data please contact api-services-support@amazon.com.
        For information about migrating to our APIs refer to our Marketplace APIs at https://developer.amazonservices.com/ref=rm_5_sv, or our Product Advertising API at https://affiliate-program.amazon.com/gp/advertising/api/detail/main.html/ref=rm_5_ac for advertising use cases.
--><!DOCTYPE html>
<html>
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
<title>Sorry! Something went wrong!</title>
<style>
  html, body {
    padding: 0;
    margin: 0
  }

  img {
    border: 0
  }

  #a {
    background: #232f3e;
    padding: 11px 11px 11px 192px
  }

  #b {
    position: absolute;
    left: 22px;
    top: 12px
  }

  #c {
    position: relative;
    max-width: 800px;
    padding: 0 40px 0 0
  }

  #e, #f {
    height: 35px;
    border: 0;
    font-size: 1em
  }


In [2]:
def get_data(no_pages):
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
    
    alls = []
    for i in range(1, no_pages+1):
        r = requests.get('https://www.amazon.com/s?k=board+games&page='+str(i), headers=headers)
        content = r.content
        soup = BeautifulSoup(content)


        for d in soup.findAll('div', attrs={'class':'sg-col-4-of-24 sg-col-4-of-12 sg-col-4-of-36 s-result-item s-asin sg-col-4-of-28 sg-col-4-of-16 sg-col sg-col-4-of-20 sg-col-4-of-32'}):
            name = d.find('span', attrs={'class':'a-size-base-plus a-color-base a-text-normal'})
            price = d.find('span', attrs={'class':'a-offscreen'})
            rating = d.find('span', attrs={'class':'a-icon-alt'})
            no_of_ratings = d.find('span', attrs={'class':'a-size-base'})
            asin = d['data-asin']

            all1=[]
            
            if asin is not None:
                all1.append(asin)
            else:
                all1.append('-')
            
            if name is not None:
                all1.append(name.text)
            else:
                all1.append("unknown-product")

            if price is not None:
                all1.append(price.text)
            else:
                all1.append('$0')

            if rating is not None:
                all1.append(rating.text)
            else:
                all1.append('-')
                
            if no_of_ratings is not None:
                all1.append(no_of_ratings.text)
            else:
                all1.append('-') 
                


            alls.append(all1)  

    return alls

In [3]:
df = pd.DataFrame(get_data(20),columns=['asin_id',
                                       'name',
                                       'price',
                                       'rating',
                                       'no_of_ratings'])

In [50]:
df

,asin_id,name,price,rating,no_of_ratings
0,B00000IVAK,SEQUENCE- Original SEQUENCE Game with Folding ...,$15.17,4.8 out of 5 stars,"11,880"
1,B076HK9H7Z,Sorry! Game,$0,4.7 out of 5 stars,"7,555"
2,B00000DMF5,Hasbro Gaming Candy Land Kingdom Of Sweet Adve...,$12.99,4.8 out of 5 stars,"7,566"
3,B07B7KS87Q,Game Of Life,$19.99,4.7 out of 5 stars,"4,284"
4,B00C0ULS3G,Battleship Classic Board Game Strategy Game Ag...,$0,4.7 out of 5 stars,"4,150"
...,...,...,...,...,...
248,B083W2L1QB,"Asmod¨¦e The Shining Board Game, Various (SH01EN)",$24.99,4.6 out of 5 stars,55
249,B0043KJW5M,7 Wonders,$0,4.8 out of 5 stars,"2,765"
250,B074L5QBBF,Game Of Thrones - Season 7,$2.99,4.8 out of 5 stars,2017
251,B07WF9LR3C,Paranormal Detectives,$35.80,4.7 out of 5 stars,77


In [51]:
df.to_csv('csv_files/amazon_items_test1.csv')

## Get Links for each Item

In [642]:
# def get_links(df_column):
#     links = []
#     for product_id in df_column:
#         if product_id != 'Missing':
#             links.append('https://www.amazon.com/product-reviews/'+str(product_id)+'/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews')
#     return links

In [643]:
# product_links = get_links(df['Asin_No'])
# product_links[0:3]

In [644]:
# def content(link):
#     page=requests.get(link,headers=headers)
#     if page.status_code==200:
#         return page
#     else:
#         return "Error" 

# Get Reviews for each Item

In [5]:
def get_reviews(asin_no, no_review_pages):
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
    
    alls = []
    for i in range(1, no_review_pages+1):
        r = requests.get('https://www.amazon.com/product-reviews/'+asin_no+'/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber='+str(i), headers=headers)
        content = r.content
        soup = BeautifulSoup(content)

        for d in soup.findAll('div', attrs={'class':'a-section review aok-relative'}):
            product_id = d['id']
            review_title = d.find('a', attrs={'class':'a-size-base a-link-normal review-title a-color-base review-title-content a-text-bold'})
            review_description = d.find('span', attrs={'class':'a-size-base review-text review-text-content'})
            rating = d.find('span', attrs={'class':'a-icon-alt'})
#             user_name = d.find('span', attrs={'class': 'a-profile-name'}).text

            all1=[]
            
            all1.append(asin_no)
            
            if product_id is not None:
                all1.append(product_id)
            else:
                all1.append("unknown-product")

            if review_title is not None:
                all1.append(review_title.text)
            else:
                all1.append('$0')

            if review_description is not None:
                #print(rating.text)
                all1.append(review_description.text)
            else:
                all1.append('-1')

            if rating is not None:
                #print(price.text)
                all1.append(rating.text)
            else:
                all1.append('0')


            alls.append(all1)  

    return alls

In [28]:
get_reviews('B08636S37Z',2)

[]

### First 100 Items (Asin_id)

In [27]:
# append reviews from all items to list

reviews = []

for id_no in df['asin_id'][0:100]:
    for row in get_reviews(id_no,2):
        reviews.append(row)

Error: [('SSL routines', 'ssl3_get_record', 'decryption failed or bad record mac')]

In [650]:
reviews

[['B076HK9H7Z',
  'R1OSPWS88F2CUZ',
  '\nDO NOT BUY!!!\n',
  '\n\n  I would give this zero stars if I could! If you’re into the original sorry game, DON’T buy this version. They changed the rules entirely while ruining the whole game! There are no longer 4 pawns per person, they only have 3! And rather than only being able to get out on a 1 or 2, you can now get out on any number.. oh and the “sorry!” card which allows you to bump someone back to start, now also has the option to move 4 places. It’s like this game was created for this new generation of entitled children that can’t take the time to finish a challenging game the right way without throwing a fit because they have instant gratification issues. Wish I opened it sooner so I could return it. I am very diasspointed in this update!!\n\n',
  '1.0 out of 5 stars'],
 ['B076HK9H7Z',
  'R1DCFJ8VYSN17B',
  '\nIs this the millennial version?\n',
  '\n\n  This is not the original sorry game. It only has 3 pawns per team instead of the 

In [651]:
df_reviews = pd.DataFrame(reviews,columns=['asin_id',
                                       'review_id',
                                       'review_title',
                                       'review_description',
                                       'rating'])

In [652]:
df_reviews

,asin_id,review_id,review_title,review_description,rating
0,B076HK9H7Z,R1OSPWS88F2CUZ,\nDO NOT BUY!!!\n,\n\n I would give this zero stars if I could!...,1.0 out of 5 stars
1,B076HK9H7Z,R1DCFJ8VYSN17B,\nIs this the millennial version?\n,\n\n This is not the original sorry game. It ...,1.0 out of 5 stars
2,B076HK9H7Z,R1V07N4GXA7RSL,\nWimp and Crybaby Edition\n,\n\n We bought this to replace our old Sorry ...,1.0 out of 5 stars
3,B076HK9H7Z,R2Z262NZDEU2EY,\nNOT the original/regular Sorry!\n,\n\n Be warned that this is not the sorry you...,2.0 out of 5 stars
4,B076HK9H7Z,RG3XIFV1PUX9Y,"\nNot the classic by a long shot, but okay.\n","\n\n Definitely not the classic game, with on...",4.0 out of 5 stars
...,...,...,...,...,...
4279,B07PXSJMHF,R2G37JKJ3LX0XX,\nGreat game\n,\n\n It's a really fun game. It's easy to le...,5.0 out of 5 stars
4280,B07PXSJMHF,R2H0T4QEAPAFI2,\nGreat family game.\n,\n\n Very well made and thick cards. Very eas...,5.0 out of 5 stars
4281,B07PXSJMHF,R1Y5NN7TY96BV3,\nThey captured minecraft in a board game.\n,\n\n You mine blocks to build or if you rathe...,5.0 out of 5 stars
4282,B07PXSJMHF,R2280MPKGGVKHF,\nGreat game\n,\n\n was wonder ful little games for children...,5.0 out of 5 stars


In [654]:
# df_reviews.to_csv('csv_files/amazon_reviews.csv')

In [33]:
df

,asin_id,name,price,rating,no_of_ratings
0,B00U26V4VQ,"Catan The Board Game, Multicolor",$0,4.8 out of 5 stars,"11,068"
1,B08636S37Z,Funko Disney The Haunted Mansion – Call of The...,$24.97,4.7 out of 5 stars,83
2,B00A2HD40E,Pandemic,$0,4.8 out of 5 stars,"7,558"
3,B07YQ641NQ,"Wingspan Board Game - A Bird-Collection, Engin...",$45.00,4.9 out of 5 stars,"1,339"
4,B07B7KS87Q,Game Of Life,$19.99,4.7 out of 5 stars,"4,284"
...,...,...,...,...,...
301,B00CYQ24QG,Goliath Pop The Pig Game — New and Improved — ...,$29.99,4.6 out of 5 stars,"3,509"
302,B086GS9VQ8,Big Size Fast Sling Puck Game for Adult and Ki...,$40.99,4.4 out of 5 stars,20
303,B07DGYX438,The Floor is Lava - Interactive Game for Kids ...,$16.97,4.5 out of 5 stars,"2,977"
304,B00004YO0E,Solid Wood Cribbage Folding Board with Playing...,$11.69,4.5 out of 5 stars,"4,968"


In [34]:
df = df.drop_duplicates(subset=['asin_id']).reset_index(drop=True)

In [ ]:
def get_reviews(asin_no, no_review_pages):
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
    
    alls = []
    for i in range(1, no_review_pages+1):
        r = requests.get('https://www.amazon.com/product-reviews/'+asin_no+'/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber='+str(i), headers=headers)
        content = r.content
        soup = BeautifulSoup(content)

        for d in soup.findAll('div', attrs={'class':'a-section review aok-relative'}):
            product_id = d['id']
            review_title = d.find('a', attrs={'class':'a-size-base a-link-normal review-title a-color-base review-title-content a-text-bold'})
            review_description = d.find('span', attrs={'class':'a-size-base review-text review-text-content'})
            rating = d.find('span', attrs={'class':'a-icon-alt'})
#             user_name = d.find('span', attrs={'class': 'a-profile-name'}).text

            all1=[]
            
            all1.append(asin_no)
            
            if product_id is not None:
                all1.append(product_id)
            else:
                all1.append("unknown-product")

            if review_title is not None:
                all1.append(review_title.text)
            else:
                all1.append('$0')

            if review_description is not None:
                #print(rating.text)
                all1.append(review_description.text)
            else:
                all1.append('-1')

            if rating is not None:
                #print(price.text)
                all1.append(rating.text)
            else:
                all1.append('0')


            alls.append(all1)  

    return alls

In [42]:
def get_prod_details(asin_no):
    
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    # get response code
    r = requests.get('https://www.amazon.com/dp/'+asin_no, headers=headers)
    content = r.content
    soup = BeautifulSoup(content)

    prod_details = []

    prod_details.append(asin_no)
    
    
    product_description = soup.find_all('div',attrs={'id':'productDescription'}).text.strip()
    prod_details.append(product_description)

    table = soup.find('table', attrs={'class':'a-keyvalue prodDetTable'})

    for line in table.findAll('tr'):
        print(line)
#         if line.find('th').text.strip() == 'Product Dimensions' or line.find('th').text.strip() == 'Package Dimensions':
#             prod_details.append(line.find('td').text.strip())

#         if line.find('th').text.strip() == 'Item Weight':
#             prod_details.append(line.find('td').text.strip())

#         if line.find('th').text.strip() == 'Manufacturer recommended age':
#             prod_details.append(line.find('td').text.strip())

#         if line.find('th').text.strip() == 'Manufacturer':
#             prod_details.append(line.find('td').text.strip())
            
    return prod_details


In [40]:
# def get_prod_details(asin_no):
    
#     headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

#     # get response code
#     r = requests.get('https://www.amazon.com/dp/'+asin_no, headers=headers)
#     content = r.content
#     soup = BeautifulSoup(content)

#     prod_details = []

#     prod_details.append(asin_no)
    
# #     product_description = soup.select('#productDescription')[0].p.get_text().strip()
#     product_description = soup.find_all('div',attrs={'id':'productDescription'})[0].text.strip()
#     prod_details.append(product_description)

#     table = soup.find('table', attrs={'class':'a-keyvalue prodDetTable'})

#     for line in table.findAll('tr'):

#         if line.find('th').text.strip() == 'Product Dimensions' or line.find('th').text.strip() == 'Package Dimensions':
#             prod_details.append(line.find('td').text.strip())

#         if line.find('th').text.strip() == 'Item Weight':
#             prod_details.append(line.find('td').text.strip())

#         if line.find('th').text.strip() == 'Manufacturer recommended age':
#             prod_details.append(line.find('td').text.strip())

#         if line.find('th').text.strip() == 'Manufacturer':
#             prod_details.append(line.find('td').text.strip())
            
#     return prod_details


In [8]:
item_descriptions = []

In [38]:
df['asin_id']

0      B00000IVAK
1      B076HK9H7Z
2      B00000DMF5
3      B07B7KS87Q
4      B00C0ULS3G
          ...    
248    B083W2L1QB
249    B0043KJW5M
250    B074L5QBBF
251    B07WF9LR3C
252    B00NQQTXU8
Name: asin_id, Length: 253, dtype: object

In [10]:
for asin_id in df['asin_id']:
    try:
        item_descriptions.append(get_prod_details(asin_id))
    except:
        pass

In [14]:
for asin_id in df['asin_id'][101:200]:
    try:
        item_descriptions.append(get_prod_details(asin_id))
    except:
        pass

In [15]:
for asin_id in df['asin_id'][201:305]:
    try:
        item_descriptions.append(get_prod_details(asin_id))
    except:
        pass